In [1]:
from keras import layers
from keras import models
from keras.optimizers import adam
import tensorflow as tf
import cv2

from DQN import Agent

import numpy as np
import gym

Using TensorFlow backend.


In [2]:
env_name = 'Breakout-v0'
env = gym.make(env_name)
np.random.seed(123)
env.seed(123)
nb_actions = env.action_space.n

In [3]:
img_height = 84
img_width = 84
img_channels = 1

state_dim = [img_height,img_width,img_channels]

In [4]:
def preprocess(observation):
    observation = cv2.cvtColor(cv2.resize(observation, (84, 110)), cv2.COLOR_BGR2GRAY)
    observation = observation[26:110,:]
    ret, observation = cv2.threshold(observation,1,255,cv2.THRESH_BINARY)
    return np.reshape(observation,(84,84,1))


In [5]:
env.reset()
action = np.random.randint(low=0, high=nb_actions)
img, reward, end_episode, info = env.step(action=action)


a = np.array(preprocess(img))
print(a.shape)

(84, 84, 1)


In [6]:
action = np.random.randint(low=0, high=nb_actions)
img, reward, end_episode, info = env.step(action=action)
b = np.array(preprocess(img))

action = np.random.randint(low=0, high=nb_actions)
img, reward, end_episode, info = env.step(action=action)
c = np.array(preprocess(img))

state = np.dstack([a, b,c])
print(state.shape)


(84, 84, 3)


In [7]:
DQN_Agent = Agent(state_dim,nb_actions)

In [8]:
DQN_Agent.model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 84, 84, 1)    0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 42, 42, 64)   3200        input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 42, 42, 64)   256         conv2d_1[0][0]                   
__________________________________________________________________________________________________
re_lu_1 (ReLU)                  (None, 42, 42, 64)   0           batch_normalization_1[0][0]      
__________________________________________________________________________________________________
max_poolin

In [9]:
batch_size = 32
n_episodes = 1000
done = False
for e in range(n_episodes): # iterate over new episodes of the game
    state = env.reset() # reset state at start of each new episode of the game
    state = preprocess(state)
    state.resize([1,84, 84,1])
    
    for time in range(5000):  # time represents a frame of the game; goal is to keep pole upright as long as possible up to range, e.g., 500 or 5000 timesteps
        env.render()
        action = DQN_Agent.act(state) # action is either 0 or 1 (move cart left or right); decide on one or other here
        next_state, reward, done, _ = env.step(action) # agent interacts with env, gets feedback; 4 state data points, e.g., pole angle, cart position        
        reward = reward if not done else -10 # reward +1 for each additional frame with pole upright        
        next_state = preprocess(next_state)
        next_state.resize([1,84, 84,1])
        DQN_Agent.remember(state, action, reward, next_state, done) # remember the previous timestep's state, actions, reward, etc.        
        state = next_state # set "current state" for upcoming iteration to the current next state        
        if done: # episode ends if agent drops pole or we reach timestep 5000
            print("episode: {}/{}, score: {}, e: {:.2}" # print the episode's score and agent's epsilon
                  .format(e, n_episodes, time, DQN_Agent.epsilon))
            break # exit loop
    if len(DQN_Agent.memory) > batch_size:
        DQN_Agent.replay(batch_size) # train the agent by replaying the experiences of the episode

episode: 0/1000, score: 243, e: 1.0
episode: 1/1000, score: 187, e: 1.0
episode: 2/1000, score: 296, e: 1.0
episode: 3/1000, score: 230, e: 1.0
episode: 4/1000, score: 250, e: 1.0
episode: 5/1000, score: 193, e: 1.0
episode: 6/1000, score: 247, e: 1.0
episode: 7/1000, score: 321, e: 1.0
episode: 8/1000, score: 182, e: 1.0
episode: 9/1000, score: 296, e: 1.0
episode: 10/1000, score: 285, e: 1.0
episode: 11/1000, score: 219, e: 0.99
episode: 12/1000, score: 224, e: 0.99
episode: 13/1000, score: 336, e: 0.99
episode: 14/1000, score: 239, e: 0.99
episode: 15/1000, score: 170, e: 0.99
episode: 16/1000, score: 214, e: 0.99
episode: 17/1000, score: 206, e: 0.99
episode: 18/1000, score: 168, e: 0.99
episode: 19/1000, score: 209, e: 0.99
episode: 20/1000, score: 220, e: 0.99
episode: 21/1000, score: 275, e: 0.99
episode: 22/1000, score: 218, e: 0.99
episode: 23/1000, score: 172, e: 0.99
episode: 24/1000, score: 234, e: 0.99
episode: 25/1000, score: 308, e: 0.99
episode: 26/1000, score: 177, e: 

episode: 214/1000, score: 381, e: 0.9
episode: 215/1000, score: 178, e: 0.9
episode: 216/1000, score: 167, e: 0.9
episode: 217/1000, score: 178, e: 0.9
episode: 218/1000, score: 270, e: 0.9
episode: 219/1000, score: 264, e: 0.9
episode: 220/1000, score: 462, e: 0.9
episode: 221/1000, score: 582, e: 0.9
episode: 222/1000, score: 215, e: 0.89
episode: 223/1000, score: 372, e: 0.89
episode: 224/1000, score: 238, e: 0.89
episode: 225/1000, score: 235, e: 0.89
episode: 226/1000, score: 273, e: 0.89
episode: 227/1000, score: 193, e: 0.89
episode: 228/1000, score: 172, e: 0.89
episode: 229/1000, score: 273, e: 0.89
episode: 230/1000, score: 251, e: 0.89
episode: 231/1000, score: 160, e: 0.89
episode: 232/1000, score: 307, e: 0.89
episode: 233/1000, score: 229, e: 0.89
episode: 234/1000, score: 304, e: 0.89
episode: 235/1000, score: 275, e: 0.89
episode: 236/1000, score: 301, e: 0.89
episode: 237/1000, score: 179, e: 0.89
episode: 238/1000, score: 275, e: 0.89
episode: 239/1000, score: 269, e:

episode: 425/1000, score: 296, e: 0.81
episode: 426/1000, score: 273, e: 0.81
episode: 427/1000, score: 304, e: 0.81
episode: 428/1000, score: 339, e: 0.81
episode: 429/1000, score: 368, e: 0.81
episode: 430/1000, score: 168, e: 0.81
episode: 431/1000, score: 230, e: 0.81
episode: 432/1000, score: 354, e: 0.81
episode: 433/1000, score: 224, e: 0.81
episode: 434/1000, score: 174, e: 0.8
episode: 435/1000, score: 230, e: 0.8
episode: 436/1000, score: 159, e: 0.8
episode: 437/1000, score: 170, e: 0.8
episode: 438/1000, score: 361, e: 0.8
episode: 439/1000, score: 439, e: 0.8
episode: 440/1000, score: 247, e: 0.8
episode: 441/1000, score: 167, e: 0.8
episode: 442/1000, score: 217, e: 0.8
episode: 443/1000, score: 376, e: 0.8
episode: 444/1000, score: 281, e: 0.8
episode: 445/1000, score: 345, e: 0.8
episode: 446/1000, score: 375, e: 0.8
episode: 447/1000, score: 316, e: 0.8
episode: 448/1000, score: 307, e: 0.8
episode: 449/1000, score: 392, e: 0.8
episode: 450/1000, score: 239, e: 0.8
epi

episode: 636/1000, score: 283, e: 0.73
episode: 637/1000, score: 233, e: 0.73
episode: 638/1000, score: 171, e: 0.73
episode: 639/1000, score: 160, e: 0.73
episode: 640/1000, score: 169, e: 0.73
episode: 641/1000, score: 171, e: 0.73
episode: 642/1000, score: 246, e: 0.73
episode: 643/1000, score: 276, e: 0.73
episode: 644/1000, score: 291, e: 0.72
episode: 645/1000, score: 186, e: 0.72
episode: 646/1000, score: 307, e: 0.72
episode: 647/1000, score: 184, e: 0.72
episode: 648/1000, score: 213, e: 0.72
episode: 649/1000, score: 431, e: 0.72
episode: 650/1000, score: 215, e: 0.72
episode: 651/1000, score: 273, e: 0.72
episode: 652/1000, score: 270, e: 0.72
episode: 653/1000, score: 182, e: 0.72
episode: 654/1000, score: 191, e: 0.72
episode: 655/1000, score: 170, e: 0.72
episode: 656/1000, score: 208, e: 0.72
episode: 657/1000, score: 234, e: 0.72
episode: 658/1000, score: 198, e: 0.72
episode: 659/1000, score: 176, e: 0.72
episode: 660/1000, score: 175, e: 0.72
episode: 661/1000, score:

episode: 847/1000, score: 220, e: 0.65
episode: 848/1000, score: 271, e: 0.65
episode: 849/1000, score: 342, e: 0.65
episode: 850/1000, score: 299, e: 0.65
episode: 851/1000, score: 340, e: 0.65
episode: 852/1000, score: 324, e: 0.65
episode: 853/1000, score: 285, e: 0.65
episode: 854/1000, score: 361, e: 0.65
episode: 855/1000, score: 340, e: 0.65
episode: 856/1000, score: 226, e: 0.65
episode: 857/1000, score: 253, e: 0.65
episode: 858/1000, score: 266, e: 0.65
episode: 859/1000, score: 163, e: 0.65
episode: 860/1000, score: 163, e: 0.65
episode: 861/1000, score: 362, e: 0.65
episode: 862/1000, score: 288, e: 0.65
episode: 863/1000, score: 269, e: 0.65
episode: 864/1000, score: 322, e: 0.65
episode: 865/1000, score: 248, e: 0.65
episode: 866/1000, score: 199, e: 0.65
episode: 867/1000, score: 429, e: 0.65


KeyboardInterrupt: 

In [ ]:
state.resize([1,84, 84,1])
DQN_Agent.model.predict(state)


In [ ]:
next_state, reward, done, _ = env.step(action)
next_state=preprocess(next_state)
next_state.shape